In [2]:
import os
import pandas as pd
import numpy as np

## Data Cleaning
- Some of the total votes counts vary for different congressional sessions. At most, this occurred in 9% of the data for each session, all the average was moreso about 5-6% of the data being corrupted. As such, I decided to overwrite all of the total_votes for each session with the number according to house.gov.
  - For the house: I deleted congressional sessions 98, 99, 100, and 101. They had no vote data.
  - For the senate: I deleted congressional sessions 80-100. They had not vote data.


In [3]:
house_fpath = '../congress-data/house/'
house_sessions = os.listdir(house_fpath)

In [7]:
house_vote_totals = {
                'house_102.csv' : 932,
                'house_103.csv' : 1122,
                'house_104.csv' : 1340,
                'house_105.csv' : 1187,
                'house_106.csv' : 1214,
                'house_107.csv' : 996,
                'house_108.csv' : 1221,
                'house_109.csv' : 1214,
                'house_110.csv' : 1876,
                'house_111.csv' : 1655,
                'house_112.csv' : 1608,
                'house_113.csv' : 1205,
                'house_114.csv' : 1327,
                'house_115.csv' : 1210,
                'house_116.csv' : 857}

senate_vote_totals = {
                'senate_101.csv' : 638,
                'senate_102.csv' : 550,
                'senate_103.csv' : 724,
                'senate_104.csv' : 919,
                'senate_105.csv' : 612,
                'senate_106.csv' : 672,
                'senate_107.csv' : 633,
                'senate_108.csv' : 675,
                'senate_109.csv' : 645,
                'senate_110.csv' : 657,
                'senate_111.csv' : 696,
                'senate_112.csv' : 486,
                'senate_113.csv' : 657,
                'senate_114.csv' : 502,
                'senate_115.csv' : 599,
                'senate_116.csv' : 565
}

In [93]:
def convert_abbrev_to_name(df, state_col):
    '''
        Converts the state abbreviations to their full name
        
        Takes a dataframe and the column name of the states
        
        Returns a dataframe with the updated information in a new column, called state_names
    '''
    states = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 
              'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 
              'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 
              'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 
              'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 
              'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 
              'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 
              'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 
              'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 
              'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 
              'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

    series = df[state_col].values

    series = np.array([states[key] if key in states.keys() else '<<>>' for key in series])
    df['state_names'] = series
    
    idx = df[df['state_names'] == '<<>>'].index
    df = df.drop(idx)

    return df

In [97]:
for sesh in house_sessions:
    fpath = '../congress-data/house/%s' % (sesh)
    df = pd.read_csv(fpath, index_col=0)
    
    df['leadership_role'] = df['leadership_role'].replace({np.nan : False})
    
    total_votes = house_vote_totals[sesh]
    df['total_votes'] = total_votes
    
    df = convert_abbrev_to_name(df, 'state')
    
    df = df.dropna(subset=['missed_votes'])
        
    df.to_csv(fpath)


In [5]:
senate_fpath = '../congress-data/senate/'
senate_sessions = os.listdir(senate_fpath)

In [6]:
senate_sessions

['senate_107.csv',
 'senate_113.csv',
 'senate_112.csv',
 'senate_106.csv',
 'senate_110.csv',
 'senate_104.csv',
 'senate_105.csv',
 'senate_111.csv',
 'senate_115.csv',
 'senate_101.csv',
 'senate_114.csv',
 'senate_102.csv',
 'senate_116.csv',
 'senate_103.csv',
 'senate_108.csv',
 'senate_109.csv']

In [98]:
for sesh in senate_sessions:
    fpath = '../congress-data/senate/%s' % (sesh)
    df = pd.read_csv(fpath, index_col=0)
    
    df['leadership_role'] = df['leadership_role'].replace({np.nan : False})
    
    total_votes = senate_vote_totals[sesh]
    df['total_votes'] = total_votes
    
    
    df = convert_abbrev_to_name(df, 'state')
    
    nans = df[df['missed_votes'].isnull()]
    if len(nans) > 0:
        print(sesh)
        for _, row in nans.iterrows():
            print(row['first_name'], row['last_name'])
    
    df = df.dropna(subset=['missed_votes'])

    #df.to_csv(fpath)